# Teil 2: Kennenlernen des Netzwerks

In dem zweiten Praxisteil werden Sie mit einem Faltungsnetz arbeiten. Hier lernen Sie, wie Sie ein Netzwerk erstellen, wie Sie Bilder durch das Netzwerk füttern wie Sie die benötigte Rechenzeit messen.

In [ ]:
"""
Dieser Code kopiert und importiert notwendige Dateien in die virtuelle Maschine von Colab.
"""
import sys, os
if 'google.colab' in sys.modules:
  if os.getcwd() == '/content':
    !git clone 'https://github.com/Criscraft/workshop_ki_hautkrebserkennung.git'
    os.chdir('workshop_ki_hautkrebserkennung')

In [ ]:
from torchsummary import summary
#from torchsummaryX import summary
import torch
import torch.nn.functional as F
import timeit
from models import Network

## Aufgabe

- Schauen Sie sich den Quellcode für das Netzwerk an (models.py)
- Wie sieht die Netzwerkarchitektur aus? Was für Schichten sind dort vorhanden?
- Nutzen Sie das Paket ```summary```, um eine Zusammenfassung über das Netzwerk zu erhalten.
- Wie viele Ausgänge besitzt das Netzwerk?
- Wie viele Parameter besitzt das Netzwerk?
- Wir werden später etwa 1000 Bilder zum Training verwenden. Was ergeben sich für Probleme?

In [ ]:
model = Network()
# Erstellung des Torch Devices (cpu). Sofern Sie eine GPU zur Verfügung haben, probieren Sie es mit 'cuda'.
device = torch.device("cpu")
# Netzwerk auf das Device transferieren
model = model.to(device)
# Geben Sie die Zusammenfassung von summary aus. Welche Eingabegröße müssen Sie für den forward pass angeben?
print(summary(model=model, input_size=(3, 128, 128), device='cpu'))
#summary(model, torch.zeros((2, 3, 128, 128)))

## 1. Forward Pass

Sehen Sie das Netzwerk jetzt in Aktion. Generieren Sie zufällige Daten mit der Funktion ```torch.rand```. Dann geben Sie es in das Netzwerk und betrachten die Ausgabe.

In [ ]:
# Instanzieren Sie das Netzwerk und transferieren Sie es auf das Pytorch Device cpu.
model = Network()
device = torch.device("cpu")
model = model.to(device)
# Erstellen Sie einen zufälligen Tensor der Ausmaße (3, 1, 128, 128) auf dem Pytorch Device.
my_tensor = torch.rand(3, 3, 128, 128, device=device)
# Berechnen Sie die Ausgabe des Netzwerkes
output = model(my_tensor)
# Geben Sie die Ausgabe aus
print(output)
print(F.softmax(output,1))

## Aufgabe

- Wir werden später einen Minibatch mit Bildern in einen Tensor verpacken. Wie ist der Tensor aufgebaut?
- Wie sind die Ausgabewerte zu interpretieren, wenn es sich um eine Klassifikationsaufgabe handelt? Tipp: Wenden Sie die Funktion ```F.softmax``` auf die Ausgabe des Netzerkes an.
- Was macht die Softmax Funktion?

## 3. Wie lange dauert der Forward Pass?

Zeit ist eine wichtige Ressource, die wir effizient nutzen wollen. Messen Sie die Zeit, die Ihr Rechner benötigt, um auf der CPU einen forward pass durchzuführen. Falls Sie eine GPU zur Verfügung haben, messen Sie auch diese Zeit.

Tipp für die Zukunft: Pytorch besitzt einen eigenen Profiler, mit dem Sie die Dauer von Tensorberechnungen auf CPU und GPU untersuchen können. Letztere sind bei anderen Profilern wie CProfiler unsichtbar.

In [ ]:
# TODO: Legen Sie die Minibatchgröße fest, sowie die Anzahl der Iterationen um einen stabilen Zeitwert zu messen
batch_size = 16
num_iterations = 50

# Setup für die Timer Klasse
setup = """\
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from models import Network

batch_size = {:d}
num_iterations = {:d}

model = Network()
device = torch.device("cpu")
model = model.to(device)
input_tensor = torch.rand(batch_size, 3, 128, 128, device=device)
""".format(batch_size, num_iterations)

# Statement für die Timer Klasse
statement = 'model(input_tensor)'

#TODO: Instanzieren Sie die Timer Klasse und berechnen Sie die mittlere Zeit für einen Forward Pass.
t = timeit.Timer(stmt=statement, setup=setup)
time_per_batch = t.timeit(number=num_iterations) / num_iterations
print("forward pass takes approx. {:.5f} seconds for one batch".format(time_per_batch))

## Aufgabe

- Wovon hängt die Schnelligkeit eines Forward Pass ab?
- Wie groß ist Ihr Geschwindigkeitsgewinn wenn Sie von der CPU auf die GPU wechseln? Wovon hängt dieser ab?
- Wie groß wäre in etwa der Geschwindigkeitsgewinn, wenn Sie von Ihrem System auf eine GPU Farm mit 128 (also sehr vielen) Grafikkarten wechseln?